
## A Sketch-based Index for Correlated Dataset Search

### Motivation

With the ever-rising amount of data available, recent researched explored queries where we want to enlarge our dataset by finding related data. This means finding the top-k tables which are joinable and correlated to our inital dataset. As these searches can be quite long they propose a more efficient and effective way of finding these tables than just naively querying over the whole data collection, which might take quite a bit. Their proposed idea has shown to achieve better results than other approaches regarding ranking accruacy and recall. 
In this notebook we want to offer you an understandable explanation of the proposed Solution and hope you are able to grasp all of it.

### Discovering Data with joinable keys and correlated data.

Steps of the algorithm:
1. Build index of tables in DB
    - Split larger tables into 2-col-tables (cross product)
    - Create a sketch of size n of each table in the Database
    - Build the index
    
    
2. Query
    - Create a sketch of the query table
    - Find correlated and joinable tables for the query-table
    

### Building the index:

1. All tables with more then 2 columns are split:
    - All numerical columns are combined with all columns containig categorical values
<br>

2. All keys k (categorical values) per table are hashed and stored as tuple with their numeric value c_k
    - <h(k), c_k>
    - For performance reason the sketch size is applied here: only the smallest n hashed values are kept
<br>

3. All categorical keys (c_k) are modified according to their hashed values (h(k))
    - If the hashed value is below or above the median of all table values, c_k is categorized in -c_k or +c_k respectively
    - <h(k), +/-c_k>
    - This is used to identify correlation
<br>

4. Picking a specific sample (=sketch) of size n per table, using n tuples with smallest hash-value
    - This way the samples are comparable


In [1]:
import os
import pandas as pd
from collections import Counter
from collections import defaultdict
from pathlib import Path
from qcr import load_index, get_kc, get_c, hash_function, build_index, key_labeling, create_sketch, cross_product_tables, print_dict

#### Step 1
In this example we want to find data correlating to the life expectancy in certain countries. To be able to search the other tables we start by building the index. Therefore, we first load 3 tables containing example data.

In [2]:
input_table = pd.read_csv('data/test_table.csv')    # import table
input_table.columns.name = 'testTable'             # assign name
table_id = input_table.columns.name                 # store name for later
display(input_table) ###Vllt weniger gut matchende Länder?

testTable,Country,Alcohol,BMI,Area(sq.mi.)
0,Armenia,3.702667,44.70625,29800.0
1,Colombia,4.419333,49.54375,1138910.0
2,Equatorial Guinea,7.342000,17.85625,28051.0
3,Germany,11.628667,51.99375,357021.0
4,Kazakhstan,6.641333,45.15625,2717300.0
5,Montenegro,2.584286,50.48750,5333.0
6,Nicaragua,3.596667,42.68125,129494.0
7,Nigeria,8.646667,19.75000,923768.0
8,Paraguay,5.527333,39.52500,406750.0
9,United States of America,8.579333,58.45000,3796742.0


In order to process tables containing multiple columns, we have to split them up into sub tables consisting of a categorical column (our key) and a numerical column (feature).

In [3]:
cat_col = get_kc(input_table)
num_col = get_c(input_table)

print_dict(cat_col, "cat_col")
print_dict(num_col, "num_col")

tables = cross_product_tables(cat_col, num_col, table_id)

table1 = tables[0] # for out example, we only build the sketch of the first table by hand
table1

cat_col:
Country 'Armenia', 'Colombia', 'Equatorial Guinea', 'Germany',
        'Kazakhstan', 'Montenegro', 'Nicaragua', 'Nigeria',
        'Paraguay', 'United States of America'
num_col:
Alcohol 3.7026666666666666, 4.419333333333333, 7.342,
        11.628666666666668, 6.641333333333334, 2.584285714285714,
        3.5966666666666667, 8.646666666666667, 5.527333333333333,
        8.579333333333333
BMI 44.70625, 49.54375, 17.85625, 51.99375, 45.15625, 50.4875,
    42.68125, 19.75, 39.525, 58.45
Area(sq.mi.) 29800.0, 1138910.0, 28051.0, 357021.0, 2717300.0, 5333.0,
             129494.0, 923768.0, 406750.0, 3796742.0


testTable_Country_Alcohol,Country,Alcohol
0,Armenia,3.702667
1,Colombia,4.419333
2,Equatorial Guinea,7.342000
3,Germany,11.628667
4,Kazakhstan,6.641333
5,Montenegro,2.584286
6,Nicaragua,3.596667
7,Nigeria,8.646667
8,Paraguay,5.527333
9,United States of America,8.579333


As you can see, the table now consists of the only categorical column of the original table, and one of the numerical columns. Also, the table name is now testTable_Country_Alcohol. The schema for this is `<originalTableName>_<categoricalColName>_<numericColName>`. This way we can later conclude which columns of which table are correlated to our query.

#

#### Step 2
Now we have tables containing only one categorical and one numerical column. <br>
We can start hashing the categorical column. For performance reasons it is advised to limit the sketch size. <br>
Although not necessary in this small case, we want to emphasize the scalability of this approach and use the limit of 5.

In [4]:
# we work on a copy of the table, to keep the variables clean for later use
hashed_table1 = table1.copy()

# create hashes
hashed_table1['hashed_keys'] = table1['Country'].map(hash_function)
print(hashed_table1)

# apply sketch size
sketch = hashed_table1.nsmallest(5, 'hashed_keys')
print('\nsketch:')
sketch

testTable_Country_Alcohol                   Country    Alcohol   hashed_keys
0                                           Armenia   3.702667 -1.040467e-39
1                                          Colombia   4.419333 -3.321928e-41
2                                 Equatorial Guinea   7.342000 -6.294744e-40
3                                           Germany  11.628667  9.140805e-40
4                                        Kazakhstan   6.641333  5.083309e-40
5                                        Montenegro   2.584286  3.881950e-40
6                                         Nicaragua   3.596667 -9.162508e-40
7                                           Nigeria   8.646667  1.278178e-39
8                                          Paraguay   5.527333  6.044821e-41
9                          United States of America   8.579333 -3.869834e-40

sketch:


testTable_Country_Alcohol,Country,Alcohol,hashed_keys
0,Armenia,3.702667,-1.040467e-39
6,Nicaragua,3.596667,-9.162508e-40
2,Equatorial Guinea,7.342000,-6.294744e-40
9,United States of America,8.579333,-3.869834e-40
1,Colombia,4.419333,-3.321928e-41


The sketch of table 1 now consists of the 5 countries whose hashed keys are lowest.

#

#### Step 3
Now the paper labels the hashed keys according to the mean of the values.<br>
To increase Readability we use the actual keys instead of the hashed keys here.<br>
Any row with its value below our mean has its key concatenated with "-1", while any row with its value above the mean has its key concatenated with "+1".

In [5]:
# 3. categorize keys by value and use as new key

# first we calculate the mean of all values of this tables numeric column
mean = sketch['Alcohol'].mean()

# again we work on a copy of the sketch, to keep the variables clean
labeled_sketch = sketch.copy()

# then label key by > median (key+1) or < median (key-1)
labeled_sketch['labeled_keys'] = [f'{key}{"+1" if value > mean else "-1"}' for key, value, hash_key in sketch.values]

print(labeled_sketch)

testTable_Country_Alcohol                   Country   Alcohol   hashed_keys  \
0                                           Armenia  3.702667 -1.040467e-39   
6                                         Nicaragua  3.596667 -9.162508e-40   
2                                 Equatorial Guinea  7.342000 -6.294744e-40   
9                          United States of America  8.579333 -3.869834e-40   
1                                          Colombia  4.419333 -3.321928e-41   

testTable_Country_Alcohol                labeled_keys  
0                                           Armenia-1  
6                                         Nicaragua-1  
2                                 Equatorial Guinea+1  
9                          United States of America+1  
1                                          Colombia-1  


As we can see Nicaragua and Armenia have been labeled "-1" and Equatorial Guinea "+1".<br>
(If the labeled_keys seem to be in a second table: this is not the case. The table is simply split into two lines, if it is too wide for the screen.)

#

#### Step 4
We finally can complete the construction of our inverted index by merging our sketches into our full inverted index. 

In [6]:
table_id = labeled_sketch.columns.name

# initialize new inverted index
inverted_index = defaultdict(set)

# now each labed key (=term) is inserted into the dict with the table name it came from
for term in labeled_sketch['labeled_keys']:
    inverted_index[term].add(table_id)

print_dict(inverted_index, "inverted index")

inverted index:
Armenia-1 {'testTable_Country_Alcohol'}
Nicaragua-1 {'testTable_Country_Alcohol'}
Equatorial Guinea+1 {'testTable_Country_Alcohol'}
United States of America+1 {'testTable_Country_Alcohol'}
Colombia-1 {'testTable_Country_Alcohol'}


As we can see the keys chosen from the table by the sketch have been tagged and are saved with a reference to the table they originate from. <br>
We now want to build the sketch for our whole database. Our Code provides the build_index function which performs the above shown procedure for a list of tables and merges them into one index.<br>
It also stores the index as a pickle file on disc. In case there is an old file stored, we remove that connection.



In [7]:
# delete old inverted index in case it exists
if os.path.exists("index.pickle"):
    Path("index.pickle").unlink()

# build index of all sub-tables of the original input-table
build_index([input_table] ,n=5)
inverted_index = load_index()
inverted_index = dict(sorted(inverted_index.items()))  # sort index for better comparison

print_dict(inverted_index, 'inverted Index')

inverted Index:
Armenia+1 {'testTable_Country_BMI'}
Armenia-1 {'testTable_Country_Alcohol',
          'testTable_Country_Area(sq.mi.)'}
Colombia+1 {'testTable_Country_BMI', 'testTable_Country_Area(sq.mi.)'}
Colombia-1 {'testTable_Country_Alcohol'}
Equatorial Guinea+1 {'testTable_Country_Alcohol'}
Equatorial Guinea-1 {'testTable_Country_BMI',
                    'testTable_Country_Area(sq.mi.)'}
Nicaragua+1 {'testTable_Country_BMI'}
Nicaragua-1 {'testTable_Country_Alcohol',
            'testTable_Country_Area(sq.mi.)'}
United States of America+1 {'testTable_Country_Alcohol',
                           'testTable_Country_BMI',
                           'testTable_Country_Area(sq.mi.)'}


This is now our inverted index for the given tables using the sketch size of 5. We can see that Armenia has an above average value in table 2, so its inhabitants have a BMI above the average of the sketched countries. On the other hand its size and Alcohol consumption is below average. <br>

So, having completed the construction of our index for the database, we can search over the inverted index using our query table. 

#

### Querying the index
As we also had tables with countries as a categorical value, we must use them here too. So we decide to search for values correlated to our initial values, having the same categorical value.

#### Step 1

We start by building the inverted index for the sketch of our query data.

In [8]:
# load query table: (key & target)
query = pd.read_csv('data/q.csv')
query.columns.name = 'q'
display(query)



q,Country,Life expectancy
0,Armenia,73.40000
1,Colombia,73.28750
2,Equatorial Guinea,55.31250
3,Germany,81.17500
4,Kazakhstan,66.76250
5,Montenegro,74.50000
6,Nicaragua,73.45000
7,Nigeria,51.35625
8,Paraguay,73.11250
9,United States of America,78.06250


As we can see this table has a categorical and a numercial column. We therefore can use it to search our inverted index. Here we want to find tables containing value correlated to our target column (Life expectancy).

In [9]:
# as above:
# 1. build sketch of query table
sketch = create_sketch(query['Country'], query['Life expectancy '], hash_function, n=5)
# 2. generate terms
search_terms = key_labeling(sketch)
print("search terms:")
search_terms

search terms:


['Armenia+1',
 'Nicaragua+1',
 'Equatorial Guinea-1',
 'United States of America+1',
 'Colombia+1']

### Negative correlation
As we also might be interested in negative correlation, we negate the values and build a second set of labeled keys.
This results in two query tables: one for positive correlation, one for negative.

In [10]:

inverse_search_terms = key_labeling(
    list(map((lambda key_value: (key_value[0], -key_value[1])), sketch))
    ) # same function as above, input key is inverted
print("inverse search terms:")
inverse_search_terms

inverse search terms:


['Armenia-1',
 'Nicaragua-1',
 'Equatorial Guinea+1',
 'United States of America-1',
 'Colombia-1']

#### Step 2
Using our new search terms, we can now query the index. We then count which table comes up how often in the results.
we limit the output to the ten most correlated tables

In [11]:

inverted_index = load_index()

result = Counter()
result.update(
    "+:" + table_id for term in search_terms for table_id in inverted_index[term]
)
result.update(
    "-:" + table_id for term in inverse_search_terms for table_id in inverted_index[term]
)

sketch = result.most_common(10)
sketch

[('+:testTable_Country_BMI', 5),
 ('-:testTable_Country_Alcohol', 4),
 ('+:testTable_Country_Area(sq.mi.)', 3),
 ('-:testTable_Country_Area(sq.mi.)', 2),
 ('+:testTable_Country_Alcohol', 1)]

### So, what do these results show us? <br>
Our first entry is of the country-bmi column combination. The "+" indicates a positive correlation, concatenated to the tableID. The 5 at the end shows that 5 entries of the sketch of this table match the query tables entries and are marked the same according to the sketches averages. This tells us, that the values are correlated (when one rises, the other does too). Therefore, the table is likely to enrich our data.<br>
For the country-alcohol column combination are 4 negative correlated entries and 1 positive correlated. Although not as clearly correlated as the first table, it might still be useful to join them to enrich our data.<br>
For the country-area column combination, there are 3 positively and 2 negatively correlated entries. This is almost a 50/50 distribution, therefore the values might just a well be random. It would be unwise to add these columns as they might not help us predict our target value. <br>
We have now decided to join our query table with the BMI column, as it's highly correlated and also join with the Alcohol column which still has a high enough chance to provide informational gain.

In [14]:
result = query.join(input_table[['Country','BMI']].set_index('Country'), on='Country' , how='left', lsuffix='_who', rsuffix='_kaggle')

result = result.join(input_table[['Country','Alcohol']].set_index('Country'), on='Country' , how='left', lsuffix='_who', rsuffix='_kaggle')

result

,Country,Life expectancy,BMI,Alcohol
0,Armenia,73.40000,44.70625,3.702667
1,Colombia,73.28750,49.54375,4.419333
2,Equatorial Guinea,55.31250,17.85625,7.342000
3,Germany,81.17500,51.99375,11.628667
4,Kazakhstan,66.76250,45.15625,6.641333
5,Montenegro,74.50000,50.48750,2.584286
6,Nicaragua,73.45000,42.68125,3.596667
7,Nigeria,51.35625,19.75000,8.646667
8,Paraguay,73.11250,39.52500,5.527333
9,United States of America,78.06250,58.45000,8.579333


### Conclusion

With our new, enriched dataset we can now proceed with our machine learning task, or which ever plans we pursue.<br>
In this notebook we showed how the approach proposed in the paper works and outlined its advantages towards naive implementations. It is noteable that the procedure takes correlation and inability into account.
Joinability of categorical columns is secured, by using these categorical columns for keys in the inverted index. Marking the distribution of the values and comparing them with the search columns numeric values ensures that the output tables & columns correlate.
 Experiments with this approach have shown to improve recall and accuracy.